In [1]:
from google import google
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
import sunlight
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood

from googleplaces import GooglePlaces, types, lang
import csv
import pandas as pd
import time
import sys
import requests
import urllib, simplejson
from pprint import pprint

In [17]:
site= 'https://www.informedimmigrant.com/organizations/#national'
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib2.Request(site,headers=hdr)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)


In [18]:
back_end = []
for a in soup.findAll("a", { "class":"organizations-nav"}):
  back_end.append(a['href'])
  
print back_end[:10]

['#national', '#alabama', '#alaska', '#arizona', '#arkansas', '#california', '#colorado', '#connecticut', '#dcmdva', '#delaware']


In [22]:
organization_names = []
organization_sites = []
organization_phones = []

for div in soup.findAll('div',{ "class":"organization"}):
  for i,strong_tag in enumerate(div.findAll('strong')):
    if i == 0:
      organization_names.append(strong_tag.text)
  for p in div.findAll('p'):
    if p.getText().startswith('Website: '):
      organization_sites.append(p.getText()[9:])
    if p.getText().startswith('Phone Number: '):
      organization_phones.append(p.getText()[14:])

In [23]:
print organization_names[:10]
print organization_sites[:10]
print organization_phones[:10]

[u'National Justice For Our Neighbors', u'World Relief', u'Define American', u'National Immigrant Justice Center', u'United We Dream (UWD)', u"America's Voice", u"The Young Center for Immigrant Children's Rights", u'National Day Laborer Organizing Network', u'Food Chain Workers Alliance', u'The TransLatin@ Coalition']
[u'njfon.org', u'www.worldrelief.org', u'https://defineamerican.com/', u'http://www.immigrantjustice.org/', u'http://unitedwedream.org/', u'http://americasvoice.org/', u'http://theyoungcenter.org/', u'http://www.ndlon.org/', u'http://foodchainworkers.org/', u'www.translatinacoalition.org ']
[u'703-766-0442', u'443-451-1900', u'N/A', u'312-660-1370', u'N/A', u'202-463-8602', u'202-794-8742', u'213-380-2783', u'213-380-4060', u'213-553-1845']


In [29]:
link = 'https://www.informedimmigrant.com/organizations/'

organization_states = []
organization_names = []
organization_sites = []
organization_phones = []

for location in back_end:
  site = link + location
  hdr = {'User-Agent': 'Mozilla/5.0'}
  req = urllib2.Request(site,headers=hdr)
  page = urllib2.urlopen(req)
  soup = BeautifulSoup(page)
  for div in soup.findAll('div',{ "class":"organization"}):
    for i,strong_tag in enumerate(div.findAll('strong')):
      if i == 0:
        organization_names.append(strong_tag.text)
        organization_states.append(location.replace('#',''))
    for p in div.findAll('p'):
      if p.getText().startswith('Website: '):
        organization_sites.append(p.getText()[9:])
      if p.getText().startswith('Phone Number: '):
        organization_phones.append(p.getText()[14:])
  


In [36]:
print len(organization_names)
print len(organization_sites)
print len(organization_phones)
print len(organization_states)

immigration_orgs = pd.DataFrame({'org_name': organization_names,
     'site': organization_sites,
     'phone': organization_phones,
    })


39052
39052
39052
39052


In [37]:
immigration_orgs.drop_duplicates(inplace = True)

In [38]:
len(immigration_orgs)

746

In [40]:
immigration_orgs.to_csv('~/w210_imwithdata/data/static_data/immigration_orgs.csv', encoding="utf-8")

In [107]:
wiki = 'https://en.wikipedia.org/wiki/List_of_LGBT_rights_organizations_in_the_United_States'
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [108]:
lgbtq_orgs = []

for a in soup.findAll('a'):
  if a.get('title','').startswith('Edit'):
    pass
  elif a.get('title','') == '':
    pass
  else:
    lgbtq_orgs.append(a.get('title',''))
  

In [109]:
print len(lgbtq_orgs)

467


In [102]:
start = lgbtq_orgs.index('Accord Alliance')
end = lgbtq_orgs.index('LGBT community')
# print lgbtq_orgs[start:end]

In [69]:
wiki = 'https://en.wikipedia.org/wiki/List_of_LGBT_community_centers_in_the_United_States'
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [80]:
lgbtq_community_orgs = []

for a in soup.findAll('a'):
  if a.get('title','').startswith('Edit'):
    pass
  elif a.get('title','') == '':
    pass
  else:
    lgbtq_community_orgs.append(a.getText())
  

In [98]:
print len(lgbtq_community_orgs)
start = lgbtq_community_orgs.index('Accord Alliance')
end = lgbtq_community_orgs.index('LGBT community')
lgbtq_comm = lgbtq_community_orgs[start:end]
lgbtq_advocacy = lgbtq_orgs[start:end]

467


In [99]:
for org in lgbtq_comm:
  lgbtq_advocacy.append(org)

In [101]:
lgbtq_organizaions = pd.DataFrame({'org_name': lgbtq_advocacy
    })
lgbtq_organizaions.drop_duplicates(inplace = True)
lgbtq_organizaions.to_csv('~/w210_imwithdata/data/static_data/lgbtq_orgs.csv', encoding="utf-8")

In [112]:
wiki = 'https://en.wikipedia.org/wiki/Category:Civil_rights_organizations_in_the_United_States'
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia
req = urllib2.Request(wiki,headers=header)
page = urllib2.urlopen(req)
soup = BeautifulSoup(page)

In [113]:
civil_rights_orgs = []

for a in soup.findAll('a'):
  if a.get('title','').startswith('Edit'):
    pass
  elif a.get('title','') == '':
    pass
  else:
    civil_rights_orgs.append(a.get('title',''))
  

In [118]:
len(civil_rights_orgs)

106

In [119]:
print len(civil_rights_orgs)
start = civil_rights_orgs.index('8th Day Center for Justice')
end = civil_rights_orgs.index('Help:Category')
print civil_rights_orgs[start:end]

106
['8th Day Center for Justice', 'ACLU of Massachusetts', 'Action for Boston Community Development', 'Alabama Christian Movement for Human Rights', 'Alaska Innocence Project', 'Alpha Suffrage Club', 'American Civil Rights Institute', 'American Equal Rights Association', 'Asian American Legal Defense and Education Fund', 'Asian Americans Advancing Justice - Los Angeles', 'Association of Community Organizations for Reform Now', 'BAMN', 'Black Action Movement', 'Board of Delegates of American Israelites', 'California Department of Fair Employment and Housing', 'Campus Pride', 'National Center for Civil and Human Rights', 'Civil Liberties Defense Center', 'Columbia Queer Alliance', 'Community Service Organization', 'Congress of Racial Equality', 'Council for United Civil Rights Leadership', 'CyberDissidents.org', 'Delta Ministry', 'Democracy Matters', 'Demos (U.S. think tank)', 'Ella Baker Center for Human Rights', 'Equal Employment Opportunity Commission', 'Equal Justice Under Law (civi

In [120]:
civil_rights_organizaions = pd.DataFrame({'org_name': civil_rights_orgs[start:end]
    })
civil_rights_organizaions.drop_duplicates(inplace = True)
civil_rights_organizaions.to_csv('~/w210_imwithdata/data/static_data/civil_rights_orgs.csv', encoding="utf-8")